<a href="https://colab.research.google.com/github/jrebull/AnaliticaPrescriptiva/blob/main/Blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¿Qué es un Block en Pyomo?
Definición:
Un Block es un contenedor jerárquico que agrupa variables, restricciones y otros componentes.
Ventajas:
- Composición modular
- Anotación estructural
- Espacios de trabajo aislados (sandboxing)
- Cada bloque tiene su propio espacio de nombres
- Se evita colisión entre componentes
- Estructura tipo árbol:
- Raíz: modelo principal
- Padre: bloque superior
- Hijos: componentes internos
Métodos útiles:
parent_component(), parent_block(), model(), component()


- Los bloques pueden ser indexados y poblados con reglas
- El primer argumento de la regla es el bloque a poblar
- Permite estructuras distintas por bloque según datos
Ejemplo:
Bloques indexados por model.T, cada uno con variables y conjuntos propios


- Los bloques pueden contener Set, Param, Constraint, etc.
- Se accede a componentes padres con model() y parent_block()
Nota:
Las reglas de componentes usan como argumento el bloque propietario

- Los datos pueden estar dispersos en la jerarquía
- Se accede iterando sobre bloques y variables
- Pyomo permite usar slices para extraer subconjuntos dinámicos



Ejemplo – Lot-Sizing Multiperíodo
Objetivo:
Determinar producción óptima $X_t$ en cada período $t \in T$ según demanda $d_t$
Variables:
- $y_t$: binaria, indica si hay producción
- $I_t$: inventario final
- $I_t^+, I_t^-$: inventario positivo y atraso
Costos:
- $c_t$: costo fijo por producir
- $h_t^+$: costo de almacenamiento
- $h_t^-$: costo por escasez




Inventario:
$I_t = I_{t-1} + X_t - d_t$
Si se permite inventario negativo:
$I_t = I_t^+ - I_t^-$
Restricción de producción:
Solo se permite si y_t = 1
Estructura del modelo:
Cada período se modela como un bloque con sus propias variables y restricciones


In [1]:
# ============================================================
# 🔧 SOLUCIÓN DEFINITIVA CON MICROMAMBA (RECOMENDADO PARA SOLVERS COIN-OR)
# ============================================================

import pyomo.environ as pyo
import os
import subprocess
import sys

print("🔧 Iniciando instalación de Pyomo, IPOPT y GLPK usando micromamba...\n")

# --- 1. Instalar micromamba (si no está) ---
MAMBA_BIN = "/usr/bin/micromamba"
if not os.path.exists(MAMBA_BIN):
    print("📦 Descargando e instalando micromamba...")
    !wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba > /dev/null 2>&1
    !mv bin/micromamba /usr/bin/micromamba
    !rm -rf bin
    print("✅ micromamba instalado.")
else:
    print("✅ micromamba ya estaba instalado.")

# --- 2. Crear entorno con Solvers ---
ENV_DIR = "/usr/local/pyomo-solvers-env"
print(f"🔧 Creando entorno conda con pyomo, ipopt y glpk en {ENV_DIR}...")

# El comando de conda-forge es más fiable
!micromamba create -y -p {ENV_DIR} -c conda-forge pyomo ipopt glpk > /dev/null 2>&1

# --- 3. Crear Enlaces Simbólicos Garantizados ---
# Forzar los ejecutables de vuelta al PATH del sistema desde el nuevo entorno.
print("🔗 Creando enlaces simbólicos garantizados...")
!ln -sf {ENV_DIR}/bin/ipopt /usr/local/bin/ipopt 2>/dev/null
!ln -sf {ENV_DIR}/bin/glpsol /usr/local/bin/glpsol 2>/dev/null
# Instalar Pyomo (por si el entorno no lo expuso bien al runtime)
!pip install -q pyomo

# --- 4. Verificación de Acceso de Pyomo ---
print("\n🔍 Verificación FINAL de Pyomo:")
try:
    # 🚨 NOTA IMPORTANTE: Usamos la ruta del symlink, que es más corta
    solver_glpk = pyo.SolverFactory('glpk', executable='/usr/local/bin/glpsol')
    solver_ipopt = pyo.SolverFactory('ipopt', executable='/usr/local/bin/ipopt')

    glpk_available = solver_glpk.available()
    ipopt_available = solver_ipopt.available()

    print(f"   ✅ Pyomo puede acceder a GLPK: {glpk_available}")
    print(f"   ✅ Pyomo puede acceder a IPOPT: {ipopt_available}")

except Exception as e:
    print(f"   ❌ Error durante la verificación final: {e}")

print("✅ Preparación completada\n")




🔧 Iniciando instalación de Pyomo, IPOPT y GLPK usando micromamba...

📦 Descargando e instalando micromamba...
✅ micromamba instalado.
🔧 Creando entorno conda con pyomo, ipopt y glpk en /usr/local/pyomo-solvers-env...
🔗 Creando enlaces simbólicos garantizados...

🔍 Verificación FINAL de Pyomo:
   ✅ Pyomo puede acceder a GLPK: True
   ✅ Pyomo puede acceder a IPOPT: True
✅ Preparación completada



In [6]:
import pyomo.environ as pyo

# ============================================================
# 1. DEFINICIÓN DEL MODELO (Lotsizing Mixto)
# ============================================================

model = pyo.ConcreteModel()
model.T = pyo.RangeSet(5) # time periods

# Parámetros
i0 = 5.0  # Inventario inicial
c = 4.6   # Costo de setup
h_pos = 0.7 # Costo de inventario
h_neg = 1.2 # Costo de escasez
P = 5.0   # Producción máxima
d = {1: 5.0, 2:7.0, 3:6.2, 4:3.1, 5:1.7} # Demanda por periodo

# Crear un bloque para cada periodo de tiempo
def lotsizing_block_rule(b, t):
    # Variables
    b.y = pyo.Var(domain=pyo.Binary)
    b.x = pyo.Var(domain=pyo.NonNegativeReals)
    b.i = pyo.Var() # Inventario/Escasez final (Real)
    b.i0 = pyo.Var()
    b.i_pos = pyo.Var(domain=pyo.NonNegativeReals)
    b.i_neg = pyo.Var(domain=pyo.NonNegativeReals)

    # Restricciones
    b.inventory = pyo.Constraint(expr=b.i == b.i0 + b.x - d[t])
    b.pos_neg = pyo.Constraint(expr=b.i == b.i_pos - b.i_neg)
    b.prod_indicator = pyo.Constraint(expr=b.x <= P * b.y)

model.lsb = pyo.Block(model.T, rule=lotsizing_block_rule)

# Restricción de enlace: Inventario inicial = Inventario final del periodo anterior
def i_linking_rule(m, t):
    if t == m.T.first():
        return m.lsb[t].i0 == i0
    return m.lsb[t].i0 == m.lsb[t-1].i

model.i_linking = pyo.Constraint(model.T, rule=i_linking_rule)

# Función objetivo: Minimizar costos
def obj_rule(m):
    return sum(c*m.lsb[t].y + h_pos*m.lsb[t].i_pos + h_neg*m.lsb[t].i_neg for t in m.T)

model.obj = pyo.Objective(rule=obj_rule)

# ============================================================
# 2. RESOLUCIÓN (USANDO LA RUTA GARANTIZADA DE GLPK)
# ============================================================

### Resolver el problema
# Usamos la ruta garantizada (asumiendo instalación exitosa)
solver = pyo.SolverFactory('glpk', executable='/usr/local/bin/glpsol')
results = solver.solve(model)
model.solutions.load_from(results)

# print the results
print("\n" + "="*70)
print("            ✨ Resultados Detallados del Problema (Por Periodo) ✨")
print("="*70)

for t in model.T:
    # 1. Producción (x)
    prod_amount = pyo.value(model.lsb[t].x)

    # 2. Inventario Positivo (i_pos)
    i_pos_val = pyo.value(model.lsb[t].i_pos)

    # 3. Setup Binario (y)
    y_val = int(round(pyo.value(model.lsb[t].y)))

    # 4. Escasez (i_neg)
    i_neg_val = pyo.value(model.lsb[t].i_neg)

    # 5. Inventario Inicial (i0)
    i0_val = pyo.value(model.lsb[t].i0)

    # 6. Inventario Final (i)
    i_val = pyo.value(model.lsb[t].i)

    # Imprimir en el formato solicitado (con claridad)
    print(f"\n--- Periodo {t} ---")
    print(f"1. Producción (x):     {prod_amount:.2f}")  # 'Period: {0}, Prod. Amount: {1}'
    print(f"2. Inv. Positivo:      {i_pos_val:.2f}")  # pydo.value(model.lsb[t].i_pos)
    print(f"3. Setup (y):          {y_val}")          # pydo.value(model.lsb[t].y)
    print(f"4. Escasez (i_neg):    {i_neg_val:.2f}") # pydo.value(model.lsb[t].i_neg)
    print(f"5. Inv. Inicial (i0):  {i0_val:.2f}")     # pydo.value(model.lsb[t].i0)
    print(f"6. Inv. Final (i):     {i_val:.2f}")      # pydo.value(model.lsb[t].i)

print("="*70)


            ✨ Resultados Detallados del Problema (Por Periodo) ✨

--- Periodo 1 ---
1. Producción (x):     3.00
2. Inv. Positivo:      3.00
3. Setup (y):          1
4. Escasez (i_neg):    0.00
5. Inv. Inicial (i0):  5.00
6. Inv. Final (i):     3.00

--- Periodo 2 ---
1. Producción (x):     5.00
2. Inv. Positivo:      1.00
3. Setup (y):          1
4. Escasez (i_neg):    0.00
5. Inv. Inicial (i0):  3.00
6. Inv. Final (i):     1.00

--- Periodo 3 ---
1. Producción (x):     5.00
2. Inv. Positivo:      -0.00
3. Setup (y):          1
4. Escasez (i_neg):    0.20
5. Inv. Inicial (i0):  1.00
6. Inv. Final (i):     -0.20

--- Periodo 4 ---
1. Producción (x):     5.00
2. Inv. Positivo:      1.70
3. Setup (y):          1
4. Escasez (i_neg):    0.00
5. Inv. Inicial (i0):  -0.20
6. Inv. Final (i):     1.70

--- Periodo 5 ---
1. Producción (x):     0.00
2. Inv. Positivo:      -0.00
3. Setup (y):          0
4. Escasez (i_neg):    0.00
5. Inv. Inicial (i0):  1.70
6. Inv. Final (i):     0.00
